Работа с объектами-контейнерами и настройка spaCy
под свои нужды
========================
05.10.2022

Основные объекты, из которых состоит API spaCy, можно разделить на две категории: контейнеры (например, объекты Token и Doc) и компоненты конвейеров обработки (например, средства частеречной разметки и распознавания именованных сущностей). В этой главе продолжим изучение объектов-контейнеров: благодаря им и их методам можно получать доступ к лингвистическим меткам, которые библиотека spaCy присваивает всем токенам в тексте.

In [28]:
import spacy as sp
nlp = sp.load('ru_core_news_lg')
input_1 = 'Здравствуйте. У меня болит горло..'
input_2 = 'Привет, как дела?'
input_3 = 'Где кошка'
input_4 = 'Хочу купить булочку'


![avatar](./doc.png)

In [29]:
from traceback import print_tb


doc = nlp(input_1)

doc_list = [doc[i] for i in range(len(doc))]   # по индексам
print(doc_list)

# можно обращаться к токенам

print([(token.head.text, token.dep_, token.text) for token in doc])




[Здравствуйте, ., У, меня, болит, горло, ..]
[('Здравствуйте', 'ROOT', 'Здравствуйте'), ('Здравствуйте', 'punct', '.'), ('меня', 'case', 'У'), ('болит', 'obl', 'меня'), ('болит', 'ROOT', 'болит'), ('болит', 'nsubj', 'горло'), ('болит', 'punct', '..')]


Пусть нам нужно найти левосторонний дочерний элемент токена в дереве синтаксических зависимостей предложения. Данная операция
позволяет найти прилагательные (при их наличии) для заданного
существительного. Этоможет понадобиться, если нужно узнать, какие
прилагательные могут модифицировать заданное существительное.

Можно также использовать атрибут Token.rights для получения правосторонних синтаксических дочерних элементов

In [30]:
print(doc[4]) # слово
print([w for w in doc[4].lefts]) # левостороннее дочернее

болит
[меня]


##### Контейнер doc.sents
doc.sents объекта Doc текст можно разделить на
отдельные предложения

In [31]:
print(input_1)
doc = nlp(input_1)

for sent in doc.sents:
    print([sent[i] for i in range(len(sent))])

Здравствуйте. У меня болит горло..
[Здравствуйте, .]
[У, меня, болит, горло, ..]


В то же время можно ссылаться на токены в состоящем из множества
предложений тексте с помощью глобальных индексов уровня документа, как показано вот здесь:

In [32]:
[doc[i] for i in range(len(doc))]

[Здравствуйте, ., У, меня, болит, горло, ..]

In [35]:
doc = nlp(u'Здравствуйте Денис. Я могу предложить вам чай')


for i,sent in enumerate(doc.sents):
    if i==1 and sent[0].pos_== 'PRON':
        print('The second sentence begins with a pronoun.')

The second sentence begins with a pronoun.


In [37]:
# сколько предложений в текстве оканчивается глаголом?
doc = nlp(u'Здравствуйте Денис.я хочу пить. Ты не хочешь. Ну и бог с тобой')

counter = 0
for sent in doc.sents:
    if sent[len(sent)-2].pos_ == 'VERB':
        counter+=1

print('Предлодений с глаглом на конце:', counter)

Предлодений с глагоом на конце: 2
